In [18]:
import pandas as pd
import random 
import json

experiment_folder = "eighth_experiment"

num_sentences_tech = 1020
num_sentences_house = 520

tech_file_path = f'data/experiments/{experiment_folder}/{num_sentences_tech}_tech_sentences_with_prompts.csv'
house_file_path = f'data/experiments/{experiment_folder}/{num_sentences_house}_house_sentences_with_prompts.csv'

tech_sentences_df = pd.read_csv(tech_file_path)
house_sentences_df = pd.read_csv(house_file_path)

num_rewrites_tech = 2115
num_rewrites_house = 1105

with open(f"data/experiments/{experiment_folder}/{num_rewrites_tech}_tech_rewrites.json", 'r') as f:
    tech_rewrites = json.load(f)

with open(f"data/experiments/{experiment_folder}/{num_rewrites_house}_house_rewrites.json", 'r') as f:
    house_rewrites = json.load(f)

tech_sentences_df

,occupation_name,skills,knowledge,prompt,original_sentences,results
0,data scientist,"['manage research data', 'publish academic res...",[],"Take this sentence from a Job Posting.\n\n""You...",You are @@intelligent@@ @@motivated@@ and @@ha...,"You are @@manage research data@@, @@publish ac..."
1,security engineer,[],"['business intelligence', 'risk management', '...","Take this sentence from a Job Posting.\n\n""##s...",##spring## ##java## ##cloud##,##business intelligence## ##risk management## ...
2,embedded system designer,"['develop creative ideas', 'verify formal spe...",[],"Take this sentence from a Job Posting.\n\n""Wil...",Will primarily @@use existing procedures and t...,Will primarily @@develop creative ideas@@ and ...
3,capacity planner,"['plan capacity', 'perform project management']",['resource description framework query language'],"Take this sentence from a Job Posting.\n\n""The...",The overall objective of your position and our...,The overall objective of your position and our...
4,chief security officer,[],[' communications protocols'],"Take this sentence from a Job Posting.\n\n""Eli...",Eligible for ##<LOCATION> Government Security ...,Eligible for understanding ##communications pr...
...,...,...,...,...,...,...
1015,software tester,"['measure software usability', 'perform softwa...",[],"Take this sentence from a Job Posting.\n\n""An ...",An ability to @@get things done quickly ( whil...,An ability to @@measure software usability (wh...
1016,software architect,[],"['computer programming', 'ASP.NET', 'STAF', 'b...","Take this sentence from a Job Posting.\n\n""##a...",##angularjs## ##sapui5## ##node.js## ##jquery#...,##computer programming## ##ASP.NET## ##STAF## ...
1017,web content manager,"['use spreadsheets software', 'comply with leg...",[],"Take this sentence from a Job Posting.\n\n""• @...",• @@Adhere to agile principles@@ and @@work cl...,• @@Use spreadsheets software@@ and @@comply w...
1018,change and configuration manager,['administer system'],"['Apache Maven', 'computer programming', 'Code...","Take this sentence from a Job Posting.\n\n""2+ ...",2+ years of experience with ##AWS## ##Linux## ...,"2+ years of experience with ##Apache Maven##, ..."


In [19]:
def convert_to_bio_tagged_sentences(sentence):
    words = sentence.replace('##', ' ## ').replace('@@', ' @@ ').split()
    tagged_words = []
    
    skill_tag = 'O'  
    knowledge_tag = 'O'  

    for word in words:
        if word == '@@':
            if skill_tag == 'O':
                skill_tag = 'B-Skill'  
            else:
                skill_tag = 'O'  
            continue  
        
        if word == '##':
            if knowledge_tag == 'O':
                knowledge_tag = 'B-Knowledge'  
            else:
                knowledge_tag = 'O'  
            continue  
        
        tagged_words.append((word, skill_tag, knowledge_tag))

        if skill_tag == 'B-Skill':
            skill_tag = 'I-Skill'
        if knowledge_tag == 'B-Knowledge':
            knowledge_tag = 'I-Knowledge'
    
    return tagged_words

In [20]:
def split_sentences_to_bio(sentences_to_split):
    splitted_sentences = []

    for sentence_to_split in sentences_to_split:
        splitted_sentences.append(convert_to_bio_tagged_sentences(sentence_to_split))
        
    return splitted_sentences

In [21]:
def append_sentences_to_conll(filename, sentences_to_append):
    with open(filename, 'a') as file:
        for sentence in sentences_to_append:
            file.write("\n")  
            for word, skill_tag, knowledge_tag in sentence:
                file.write(f"{word}\t{skill_tag}\t{knowledge_tag}\n")


In [22]:
bio_splitted_sentences = split_sentences_to_bio(tech_sentences_df['results'])
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_tech_train.conll", bio_splitted_sentences)

In [23]:
bio_splitted_sentences = split_sentences_to_bio(house_sentences_df['results'])
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_house_train.conll", bio_splitted_sentences)

In [24]:
bio_splitted_sentences = split_sentences_to_bio(tech_rewrites)
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_tech_train.conll", bio_splitted_sentences)

In [25]:
bio_splitted_sentences = split_sentences_to_bio(house_rewrites)
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_house_train.conll", bio_splitted_sentences)